### OIL Factor

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

price = pd.read_parquet(
    '../Data/US_stock_Price.parquet',
).resample('W-FRI').last()

In [18]:
oil_price = pd.read_excel(
    '../Data/WTI.xlsx',
    index_col = 0
).resample('W-FRI').last()

In [19]:
oil_ret = oil_price.pct_change()

In [20]:
oil_ret.loc['1990':].index

DatetimeIndex(['1990-01-05', '1990-01-12', '1990-01-19', '1990-01-26',
               '1990-02-02', '1990-02-09', '1990-02-16', '1990-02-23',
               '1990-03-02', '1990-03-09',
               ...
               '2024-08-30', '2024-09-06', '2024-09-13', '2024-09-20',
               '2024-09-27', '2024-10-04', '2024-10-11', '2024-10-18',
               '2024-10-25', '2024-11-01'],
              dtype='datetime64[ns]', name='Date', length=1818, freq='W-FRI')

In [62]:
price = pd.read_parquet(
    '../Data/US_stock_Price.parquet',
)

In [63]:
price_ret = price.pct_change()

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_20240/1882691314.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  price_ret = price.pct_change()


In [64]:
price_ret.index

DatetimeIndex(['1990-01-02', '1990-01-03', '1990-01-04', '1990-01-05',
               '1990-01-08', '1990-01-09', '1990-01-10', '1990-01-11',
               '1990-01-12', '1990-01-15',
               ...
               '2024-10-18', '2024-10-21', '2024-10-22', '2024-10-23',
               '2024-10-24', '2024-10-25', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31'],
              dtype='datetime64[ns]', name='Date', length=8776, freq=None)

In [25]:
oil_excess = price_ret - oil_ret.loc['1990':].values

In [65]:
from tqdm import tqdm
up_30per = dict()
down_30per = dict()

mean_excess = pd.DataFrame(
    index = price_ret.index,
    columns = ['Up 30per', 'Down 30per']
)

for i in tqdm(zip([f'{year}-06' for year in range(1990, 2025)], [f'{year}-05' for year in range(1991, 2026)])) :
    length = int(len(oil_excess.loc[i[0]:i[1]].mean().sort_values(ascending = False).dropna()) * 0.3)
    up_30per[i[0]] = oil_excess.loc[i[0]:i[1]].mean().sort_values(ascending = False).dropna().iloc[:length].index.tolist()
    down_30per[i[0]] = oil_excess.loc[i[0]:i[1]].mean().sort_values(ascending = True).dropna().iloc[:length].index.tolist()
    
    mean_excess.loc[i[0]:i[1], 'Up 30per'] = price_ret.loc[i[0]:i[1], up_30per[i[0]]].mean(axis = 1).values
    mean_excess.loc[i[0]:i[1], 'Down 30per'] = price_ret.loc[i[0]:i[1], down_30per[i[0]]].mean(axis = 1).values

35it [00:02, 15.77it/s]


In [67]:
oil_factor = mean_excess['Up 30per'] - mean_excess['Down 30per']

In [69]:
pd.DataFrame(oil_factor).to_parquet('../Data/OIL_Factor.parquet')